In [1]:
import sys 
import os
import math
from utils import colorstr, LOGGER
sys.path.append(os.path.abspath("pytorch-image-models"))
from timm import create_model
from timm.data.mixup import Mixup
from fastai.vision.all import *
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import mean_squared_error

set_seed(1, reproducible=True)

In [2]:
train_df = pd.read_csv('train.csv')
train_df['path'] = train_df['Id'].map(lambda x:'train/'+ str(x) +'.jpg')
train_df = train_df.drop(columns=['Id'])
train_df = train_df.sample(frac=1).reset_index(drop=True) #shuffle dataframe
print(f"There are {len(train_df)} images")
train_df['Pawpularity'].hist(figsize = (10, 5))
print(f"The mean Pawpularity score is {train_df['Pawpularity'].mean()}")
print(f"The median Pawpularity score is {train_df['Pawpularity'].median()}")
print(f"The standard deviation of the Pawpularity score is {train_df['Pawpularity'].std()}")
print(f"There are {len(train_df['Pawpularity'].unique())} unique values of Pawpularity score")
train_df['norm_score'] = train_df['Pawpularity']/100

There are 9912 images
The mean Pawpularity score is 38.03904358353511
The median Pawpularity score is 33.0
The standard deviation of the Pawpularity score is 20.59199010577444
There are 100 unique values of Pawpularity score


In [3]:
seed=12
set_seed(seed, reproducible=True)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.use_deterministic_algorithms = True

In [4]:
#Sturges' rule
num_bins = int(np.floor(1+np.log2(len(train_df))))
print(num_bins)

train_df['bins'] = pd.cut(train_df['norm_score'], bins=num_bins, labels=False)
train_df['bins'].hist()

14


<AxesSubplot:>

In [5]:
def petfinder_rmse(input,target):
    return 100*torch.sqrt(F.mse_loss(F.sigmoid(input.flatten()), target))

def get_data(fold):
    train_df_f = train_df.copy()
    # add is_valid for validation fold
    train_df_f['is_valid'] = (train_df_f['fold'] == fold)
    splitter = RandomSplitter(0.2)
    # Change RandomSplitter to IndexSplitter
    splitter = IndexSplitter(train_df_f[train_df_f['is_valid']].index.values)
    dls = DataBlock(blocks=(ImageBlock, RegressionBlock),
                get_x=ColReader('path'),
                get_y=ColReader('norm_score'),
                splitter=splitter,
                item_tfms=Resize(224), #pass in item_tfms
                batch_tfms=setup_aug_tfms([Brightness(), 
                                           Contrast(), 
                                           Hue(), 
                                           Saturation()])
               )
    
    paw_dls = dls.dataloaders(train_df_f, 
                          bs=BATCH_SIZE,
                          num_workers=8,
                          seed=seed)
    
    return paw_dls, splitter


def get_learner(fold_num):
    data, splitter = get_data(fold_num)
    model = create_model('swin_large_patch4_window7_224', 
                         pretrained=True, 
                         num_classes=data.c)
    learn = Learner(data, model, 
                    loss_func=BCEWithLogitsLossFlat(), 
                    metrics=petfinder_rmse, 
                    cbs=[MixUp(0.25)]).to_fp16()
    
    return learn, splitter

In [6]:
train_df['fold'] = -1
N_FOLDS = 10
BATCH_SIZE = 4

strat_kfold = StratifiedKFold(n_splits=N_FOLDS, random_state=seed, shuffle=True)
for i, (_, train_index) in enumerate(strat_kfold.split(train_df.index, train_df['bins'])):
    train_df.iloc[train_index, -1] = i
    
train_df['fold'] = train_df['fold'].astype('int')

    
    
learn, splitter = get_learner(fold_num=0)
learn.lr_find()

/home/umbra/anaconda3/envs/MyTorch/lib/python3.8/site-packages/torch/functional.py:445: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  /opt/conda/conda-bld/pytorch_1634272128894/work/aten/src/ATen/native/TensorShape.cpp:2157.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
Loading pretrained weights from url (https://github.com/SwinTransformer/storage/releases/download/v1.0.0/swin_large_patch4_window7_224_22kto1k.pth)


SuggestedLRs(valley=0.0005754399462603033)

In [7]:
learn.fit_one_cycle(5, 
                    2e-5, 
                    cbs=[SaveModelCallback(), 
                         EarlyStoppingCallback(monitor='petfinder_rmse', 
                                               comp=np.less, 
                                               patience=2)]) 


epoch,train_loss,valid_loss,petfinder_rmse,time
0,0.652484,0.645134,16.516209,10:32
1,0.650493,0.647341,16.805586,10:18
2,0.640004,0.642100,15.949996,08:54
3,0.628254,0.642803,16.032101,10:05
4,0.637023,0.642060,16.009418,10:26


/home/umbra/anaconda3/envs/MyTorch/lib/python3.8/site-packages/torch/nn/functional.py:1806: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


Better model found at epoch 0 with valid_loss value: 0.645134449005127.
Better model found at epoch 2 with valid_loss value: 0.642099916934967.
Better model found at epoch 4 with valid_loss value: 0.6420602202415466.
No improvement since epoch 2: early stopping


In [9]:
dir(learn)

['T_destination',
 '__call__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__enter__',
 '__eq__',
 '__exit__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__stored_args__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_apply',
 '_backward_hooks',
 '_bn_bias_state',
 '_buffers',
 '_call_impl',
 '_call_one',
 '_component_attr_filter',
 '_default',
 '_dir',
 '_do_epoch',
 '_do_epoch_train',
 '_do_epoch_validate',
 '_do_fit',
 '_do_one_batch',
 '_end_cleanup',
 '_forward_hooks',
 '_forward_pre_hooks',
 '_get_backward_hooks',
 '_get_name',
 '_grab_cbs',
 '_is_full_backward_hook',
 '_load_from_state_dict',
 '_load_state_dict_pre_hooks',
 '_maybe_warn_non_full_backward_hook',
 '_metrics',
 '_modules',
 '_named_members',
 '_non_persi

In [22]:
learn.yb

(None,)